In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA sales_bronze;

select current_catalog(), current_schema();

current_catalog(),current_schema()
workspace,sales_bronze


In [0]:
%sql
DESCRIBE SCHEMA EXTENDED sales_bronze

database_description_item,database_description_value
Catalog Name,workspace
Namespace Name,sales_bronze
Comment,
Location,
Owner,abhilashaalexander93@gmail.com
Properties,
Predictive Optimization,ENABLE (inherited from METASTORE metastore_aws_us_east_2)


In [0]:
%sql
SHOW VOLUMES;

database,volume_name
sales_bronze,sales


In [0]:
spark.sql(f"LIST '/Volumes/workspace/sales_bronze/sales'").display()

path,name,size,modification_time
/Volumes/workspace/sales_bronze/sales/sales_data.csv,sales_data.csv,15240280,1765127411000


In [0]:
spark.sql(f'''select * from text.`/Volumes/workspace/sales_bronze/sales` limit 10
          ''').display()

value
"Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue"
"2013-11-26,26,November,2013,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950"
"2015-11-26,26,November,2015,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950"
"2014-03-23,23,March,2014,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,23,45,120,1366,1035,2401"
"2016-03-23,23,March,2016,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,20,45,120,1188,900,2088"
"2014-05-15,15,May,2014,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,238,180,418"
"2016-05-15,15,May,2016,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,5,45,120,297,225,522"
"2014-05-22,22,May,2014,47,Adults (35-64),F,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,199,180,379"
"2016-05-22,22,May,2016,47,Adults (35-64),F,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,2,45,120,100,90,190"
"2014-02-22,22,February,2014,35,Adults (35-64),M,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,22,45,120,1096,990,2086"


In [0]:
%sql
SELECT *
FROM read_files (
'/Volumes/workspace/' || 'sales_bronze' || '/sales/',
format => 'csv',
header => 'true',
inferSchema =>  'true'
) limit 10;

Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue,_rescued_data
2013-11-26,26,November,2013,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950,null
2015-11-26,26,November,2015,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950,null
2014-03-23,23,March,2014,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,23,45,120,1366,1035,2401,null
2016-03-23,23,March,2016,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,20,45,120,1188,900,2088,null
2014-05-15,15,May,2014,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,238,180,418,null
2016-05-15,15,May,2016,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,5,45,120,297,225,522,null
2014-05-22,22,May,2014,47,Adults (35-64),F,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,199,180,379,null
2016-05-22,22,May,2016,47,Adults (35-64),F,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,2,45,120,100,90,190,null
2014-02-22,22,February,2014,35,Adults (35-64),M,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,22,45,120,1096,990,2086,null
2016-02-22,22,February,2016,35,Adults (35-64),M,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,21,45,120,1046,945,1991,null


In [0]:
%sql
DROP TABLE IF EXISTS sales_bronze.current_sales;
USE SCHEMA sales_bronze;
CREATE TABLE current_sales AS
SELECT
Date,
Customer_Age,
Customer_Gender,
Country,
Product_Category,
Sub_Category,
Product,
Order_Quantity,
Profit,
Cost,
Revenue
FROM read_files (
  '/Volumes/workspace/' || 'sales_bronze' || '/sales/',
  format => 'csv',
  header => 'true',
  inferSchema => 'true'
) where Sub_Category in ('Gloves','Vests') limit 500 ;

select distinct Product_Category, Sub_Category, Product from sales_bronze.current_sales;

Product_Category,Sub_Category,Product
Clothing,Gloves,Fender Set - Mountain
Clothing,Gloves,"Half-Finger Gloves, L"
Clothing,Gloves,"Half-Finger Gloves, M"
Clothing,Gloves,"Half-Finger Gloves, S"


In [0]:
spark.catalog.listTables(f"workspace.sales_bronze")

[Table(name='current_sales', catalog='workspace', namespace=['sales_bronze'], description=None, tableType='MANAGED', isTemporary=False)]

In [0]:
%sql
DESCRIBE EXTENDED sales_bronze.current_sales;

col_name,data_type,comment
Date,date,null
Customer_Age,int,null
Customer_Gender,string,null
Country,string,null
Product_Category,string,null
Sub_Category,string,null
Product,string,null
Order_Quantity,int,null
Profit,int,null
Cost,int,null


In [0]:
%sql
DESCRIBE HISTORY current_sales

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2025-12-07T17:47:50.000Z,70858962676464,abhilashaalexander93@gmail.com,CREATE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(4463430101549251),1207-172218-fcwcn3vc-v2n,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 500, numOutputBytes -> 9214)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13


In [0]:
%sql
select * from sales_bronze.current_sales version as of 0
limit 10;

Date,Customer_Age,Customer_Gender,Country,Product_Category,Sub_Category,Product,Order_Quantity,Profit,Cost,Revenue
2013-11-01,29,F,United States,Clothing,Gloves,Fender Set - Mountain,18,165,144,309
2015-11-01,29,F,United States,Clothing,Gloves,Fender Set - Mountain,16,147,128,275
2014-02-20,29,F,United States,Clothing,Gloves,Fender Set - Mountain,2,18,16,34
2016-02-20,29,F,United States,Clothing,Gloves,Fender Set - Mountain,1,9,8,17
2013-09-28,51,M,Australia,Clothing,Gloves,"Half-Finger Gloves, L",6,71,54,125
2015-09-28,51,M,Australia,Clothing,Gloves,"Half-Finger Gloves, L",6,71,54,125
2014-05-01,47,F,Australia,Clothing,Gloves,"Half-Finger Gloves, L",1,12,9,21
2016-05-01,47,F,Australia,Clothing,Gloves,"Half-Finger Gloves, L",3,36,27,63
2013-08-07,44,F,Australia,Clothing,Gloves,"Half-Finger Gloves, M",13,154,117,271
2015-08-07,44,F,Australia,Clothing,Gloves,"Half-Finger Gloves, M",10,119,90,209
